# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [22]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [46]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

In [78]:
spaceship.shape

(8693, 14)

In [50]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [80]:
spaceship2 = spaceship.dropna()

In [82]:
spaceship2.shape

(6606, 14)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [84]:
#your code here
features = spaceship2.drop(columns = ["HomePlanet","CryoSleep", "Cabin", "Destination", "VIP", "Name", "Transported", "PassengerId"])
target = spaceship2["Age"]

**Perform Train Test Split**

In [86]:
#your code here
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [88]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [90]:
X_train_norm_np = normalizer.transform(X_train)

X_test_norm_np = normalizer.transform(X_test)

In [92]:
X_train_norm_df = pd.DataFrame(X_train_norm_np, columns = X_train.columns, index=X_train.index)
X_train_norm_df.head()

Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck
3432  0.405063      0.00000   0.000000      0.000000  0.000000  0.000000
7312  0.050633      0.00000   0.000000      0.000000  0.000000  0.000000
2042  0.379747      0.00000   0.007916      0.000000  0.051276  0.000000
4999  0.215190      0.00131   0.000000      0.046111  0.016378  0.000049
5755  0.329114      0.00000   0.000000      0.000000  0.000000  0.000000

In [94]:
X_test_norm_df = pd.DataFrame(X_test_norm_np, columns = X_test.columns, index=X_test.index)
X_test_norm_df.head()

Age  RoomService  FoodCourt  ShoppingMall      Spa  VRDeck
2453  0.632911          0.0   0.000000           0.0  0.00000     0.0
1334  0.227848          0.0   0.000000           0.0  0.00000     0.0
8272  0.189873          0.0   0.000000           0.0  0.00000     0.0
5090  0.658228          0.0   0.000000           0.0  0.00000     0.0
4357  0.784810          0.0   0.054775           0.0  0.07774     0.0

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [96]:
#your code here
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100, # number of models to use
                               max_samples = 1000)

In [98]:
bagging_reg.fit(X_train_norm_df, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [101]:
y_pred_test_bag = bagging_reg.predict(X_test_norm_df)

print(f"MAE {mean_absolute_error(y_pred_test_bag, y_test): .2f}")
print(f"RMSE {mean_squared_error(y_pred_test_bag, y_test, squared=False): .2f}")
print(f"R2 score {bagging_reg.score(X_test_norm_df, y_test): .2f}")

MAE  0.02
RMSE  0.21
R2 score  1.00


C:\Users\vasco\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Random Forests

In [103]:
#your code here
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [107]:
forest.fit(X_train_norm_df, y_train)

RandomForestRegressor(max_depth=20)

In [109]:
y_pred_test_rf = forest.predict(X_test_norm_df)

print(f"MAE, {mean_absolute_error(y_pred_test_rf, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_rf, y_test, squared=False): .2f}")
print(f"R2 score, {forest.score(X_test_norm_df, y_test): .2f}")

MAE,  0.00
RMSE,  0.05
R2 score,  1.00


C:\Users\vasco\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [117]:
#your code here
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [119]:
gb_reg.fit(X_train_norm_df, y_train)

GradientBoostingRegressor(max_depth=20)

In [121]:
y_pred_test_gb = gb_reg.predict(X_test_norm_df)

print(f"MAE, {mean_absolute_error(y_pred_test_gb, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_gb, y_test, squared=False): .2f}")
print(f"R2 score, {gb_reg.score(X_test_norm_df, y_test): .2f}")

MAE,  0.00
RMSE,  0.03
R2 score,  1.00


C:\Users\vasco\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [111]:
#your code here
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [113]:
ada_reg.fit(X_train_norm_df, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [115]:
y_pred_test_ada = ada_reg.predict(X_test_norm_df)

print(f"MAE, {mean_absolute_error(y_pred_test_ada, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_ada, y_test, squared=False): .2f}")
print(f"R2 score, {ada_reg.score(X_test_norm_df, y_test): .2f}")

MAE,  0.00
RMSE,  0.05
R2 score,  1.00


C:\Users\vasco\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

In [ ]:
#comment here
#Bagging and Pasting because even thought R2 is always 1 RMSE is higher than them all